## Steinhardt's bond orientational order parameters

Steinhardt's bond orientational order parameters are a set of parameters based on the local atomic environment. These parameters have been used extensively for various uses such as distinction of crystal structures, identification of solid and liquid atoms and identification of defects.

These parameters, which are rotationally and translationally invariant are defined by,


$$ q_l (i) =  \Big(  \frac{4\pi}{2l+1}  \sum_{m=-l}^l | q_{lm}(i) |^2 \Big )^{\frac{1}{2}} $$ 

where, 

$$ q_{lm} (i) =  \frac{1}{N(i)} \sum_{j=1}^{N(i)} Y_{lm}(\pmb{r}_{ij}) $$ 

in which $Y_{lm}$ are the spherical harmonics and $N(i)$ is the number of neighbours of particle $i$, $\pmb{r}_{ij}$ is the vector connecting particles $i$ and $j$, and $l$ and $m$ are both intergers with $m \in [-l,+l]$. Various parameters have found specific uses, such as $q_2$ and $q_6$ for identification of crystallinity, $q_6$ for identification of solidity, and $q_4$ and $q_6$ for distinction of crystal structures. The traditional method makes use of a cutoff radius which takes into account the neighbors of an atom. 

#### Finding the neighbors using a cutoff

The most common method to calculate the neighbors of an atom is using a cutoff radius. The neighborhood of an atom for calculation of Steinhardt's parameters is often carried out using this method. A cutoff is selected based on the properties of the system, one of the common methods is to select the cutoff in a way that it coincides with the first minimum of the radial distribution function. Once a cutoff is decided, the neighbors of an atom are those that fall within this selected radius. The following code snippet will use the cutoff method to calculate neighbors. Please check the examples section of basic use of the module. ``conf.dump`` is assumed to be the input file containing simulation snapshot. A cutoff radius of 3 is assumed for calculation of neighbors.

In [ ]:
import pybop.core as pc
sys = pc.System()
sys.read_inputfile('conf.dump')
sys.get_neighbors(method='cutoff', cutoff=3)

#### Calculating Steinhardt's parameters using the module

The module can be used easily for the calculation of $q_l$ parameters. Once the neighbors have been calculated using the above code snippet, the $q_l$ can be calculated by, for example, $q_4$ and $q_6$ by,

In [ ]:
sys.calculate_q([4, 6])
q = sys.get_qvals([4, 6])

### Averaged Steinhardt's parameters

At high temperatures, thermal vibrations affect the atomic positions. This in turn leads to overlapping distributions of $q_l$ parameters, which in turn leads to difficulty in identification of crystal structures. In order to address this problem, the averaged version $\bar{q}_l$ of Steinhardt's parameters was introduced by Lechner and Dellago. $\bar{q}_l$ is given by,

$$ \bar{q}_l (i) =  \Big(  \frac{4\pi}{2l+1}  \sum_{m=-l}^l \Big| \frac{1}{\tilde{N}(i)} \sum_{k=0}^{\tilde{N}(i)} q_{lm}(k) \Big|^2 \Big )^{\frac{1}{2}} $$ 

where the sum from $k=0$ to $\tilde{N}(i)$ is over all the neighbors and the particle itself. The averaged parameters takes into account the first neighbor shell and also information from the neighboring atoms and thus reduces the overlap between the distributions. 


#### Calculation using the module

Assuming that the system is set up correctly as shown in the previous section, the averaged versions can be calculated by using the ``averaged = True`` keyword.

In [ ]:
sys.calculate_q([4, 6], averaged=True)
q = sys.get_qvals([4, 6], averaged=True)

## Towards a parameter-free description

### Adaptive cutoff methods

Selecting a hard cutoff radius gives rise to some problems such as-  

* Atomic vibrations due to temperature makes the selection of a cutoff difficult.  
* If there is more than one structure present in the system, for example, bcc and fcc, the selection of cutoff such that it includes the first shell of both structures can be difficult.  

With the aim of rectifying these problems, various adaptive approaches have been proposed. Two of the methods implemented in the module is discussed below. 

#### Solid angle based nearest neighbor algorithm (SANN)

SANN algorithm determines the cutoff radius by counting the solid angles around an atom and equating it to $4\pi$. The algorithm solves the following equation iteratively.  
$$R_i^{(m)} = \frac{\sum_{j=1}^m r_{i,j}}{m-2} < r_{i, m+1}$$  
where $i$ is the host atom, $j$ are it's neighbors. $R_i$ is the cutoff radius for each particle $i$ which is found by increasing the neighbor of neighbors $m$. For a description of the algorithm and more details, please check. SANN algorithm can be used to find the neighbors by,  

In [ ]:
import pybop.core as pc
sys = pc.System()
sys.read_inputfile('conf.dump')
sys.get_neighbors(method='cutoff', cutoff='sann')

Since SANN algorithm involves sorting, a sufficiently large cutoff is used in the beginning to reduce the number entries to be sorted. This parameter is calculated by,  
$$ r_{initial} = \mathrm{threshold} \times \bigg(\frac{\mathrm{Simulation~box~volume}}{\mathrm{Number~of~particles}}\bigg)^{\frac{1}{3}}$$
a tunable ``threshold`` parameter can be set through function arguments.

#### Adaptive cutoff method

An adaptive cutoff specific for each atom can also be found using an algorithm similar to adaptive common neighbor analysis. This adaptive cutoff is calculated by first making a list of all neighbor distances for each atom similar to SANN method. Once this list is available, then the cutoff is calculated from,  
$$ r_{cut}(i) = \mathrm{padding}\times \bigg(\frac{1}{\mathrm{nlimit}} \sum_{j=1}^{\mathrm{nlimit}} r_{ij} \bigg)$$

This method can be chosen by,

In [ ]:
import pybop.core as pc
sys = pc.System()
sys.read_inputfile('conf.dump')
sys.get_neighbors(method='cutoff', cutoff='adaptive')

The ``padding`` and ``nlimit`` parameters in the above equation can be tuned using the respective keywords.

Either of the adaptive method can be used to find neighbors, which can then be used to calculate Steinhardt's parameters or averaged versions.

### Voronoi tessellation

Voronoi tessellation provides a completely parameter free geometric approach for calculation of neighbors. ``Voro++`` code is used for Voronoi tessellation. Neighbors can be calculated using this method by,

In [ ]:
import pybop.core as pc
sys = pc.System()
sys.read_inputfile('conf.dump')
sys.get_neighbors(method='voronoi')